In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bitfest-datathon-2025/sample_submission.csv
/kaggle/input/bitfest-datathon-2025/train.csv
/kaggle/input/bitfest-datathon-2025/test.csv


let's import necessary libraries and files.

In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


file_path = '/kaggle/input/bitfest-datathon-2025/train.csv'
df = pd.read_csv(file_path)


print(df.head())




  address                                   career_objective  \
0     NaN  Big data analytics working and database wareho...   
1     NaN  Fresher looking to join as a data analyst and ...   
2     NaN                                                NaN   
3     NaN  To obtain a position in a fast-paced business ...   
4     NaN  Professional accountant with an outstanding wo...   

                                              skills  \
0  ['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...   
1  ['Data Analysis', 'Data Analytics', 'Business ...   
2  ['Software Development', 'Machine Learning', '...   
3  ['accounts payables', 'accounts receivables', ...   
4  ['Analytical reasoning', 'Compliance testing k...   

                        educational_institution_name  \
0  ['The Amity School of Engineering & Technology...   
1  ['Delhi University - Hansraj College', 'Delhi ...   
2    ['Birla Institute of Technology (BIT), Ranchi']   
3  ['Martinez Adult Education, Business Training ...  

we're only considering relevant columns and using one-hot encoding for categorical columns.

In [15]:
relevant_columns = [
     'career_objective','educational_results','skills',  
    'degree_names', 'passing_years', 'major_field_of_studies', 
    'professional_company_names', 'related_skils_in_job', 
    'positions', 'responsibilities', 'matched_score','skills_required'
]

df = df[relevant_columns]


print(df.columns)


df.fillna('Unknown', inplace=True)


df.drop_duplicates(inplace=True)


df['passing_years'] = df['passing_years'].apply(lambda x: x[0] if isinstance(x, list) else x)


X = df.drop('matched_score', axis=1)
y = df['matched_score']


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

text_features = [col for col in categorical_features if col not in ['Unknown']]
text_transformers = [(feature, TfidfVectorizer(), feature) for feature in text_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, [col for col in categorical_features if col not in text_features])
    ] + text_transformers)


X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)


'''model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


test_file_path = 'c:/Users/User/Desktop/kuet/test.csv'
test_df = pd.read_csv(test_file_path)


test_df.fillna('Unknown', inplace=True)


test_df = test_df.drop(columns=['address', 'company_urls', 'extra_curricular_organization_links', 'online_links'], errors='ignore')


test_df['passing_years'] = test_df['passing_years'].apply(lambda x: x[0] if isinstance(x, list) else x)

X_test_final = test_df.drop('ID', axis=1)  


X_test_final = preprocessor.transform(X_test_final)


test_predictions = model.predict(X_test_final)


predictions_df = pd.DataFrame({
    'ID': test_df['ID'],
    'matched_score': test_predictions
})


predictions_df.to_csv('c:/Users/User/Desktop/kuet/test_predictions.csv', index=Falsse)'''

Index(['career_objective', 'educational_results', 'skills', 'degree_names',
       'passing_years', 'major_field_of_studies', 'professional_company_names',
       'related_skils_in_job', 'positions', 'responsibilities',
       'matched_score', 'skills_required'],
      dtype='object')


"model = RandomForestRegressor(random_state=42)\nmodel.fit(X_train, y_train)\n\n\ny_pred = model.predict(X_test)\n\n\nmse = mean_squared_error(y_test, y_pred)\nprint(f'Mean Squared Error: {mse}')\n\n\ntest_file_path = 'c:/Users/User/Desktop/kuet/test.csv'\ntest_df = pd.read_csv(test_file_path)\n\n\ntest_df.fillna('Unknown', inplace=True)\n\n\ntest_df = test_df.drop(columns=['address', 'company_urls', 'extra_curricular_organization_links', 'online_links'], errors='ignore')\n\n\ntest_df['passing_years'] = test_df['passing_years'].apply(lambda x: x[0] if isinstance(x, list) else x)\n\nX_test_final = test_df.drop('ID', axis=1)  \n\n\nX_test_final = preprocessor.transform(X_test_final)\n\n\ntest_predictions = model.predict(X_test_final)\n\n\npredictions_df = pd.DataFrame({\n    'ID': test_df['ID'],\n    'matched_score': test_predictions\n})\n\n\npredictions_df.to_csv('c:/Users/User/Desktop/kuet/test_predictions.csv', index=Falsse)"

In [ ]:
'''import optuna
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error

def objective(trial):
    cat_params = dict(
        iterations=5000,  
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0, 1.5),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        border_count=trial.suggest_int("border_count", 32, 255),
        task_type='GPU',
        early_stopping_rounds=200,
        verbose=False
    )
    
    model = CatBoostRegressor(**cat_params)
    X_train_pool = Pool(X_train, y_train)
    X_valid_pool = Pool(X_val, y_val)
    model.fit(X=X_train_pool, eval_set=X_valid_pool)
    
    y_pred = model.predict(X_val)
    score = mean_squared_error(y_val, y_pred, squared=False)
    
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  
best_params = study.best_params'''

CatBoostRegressor is used by our team.

In [16]:
import optuna
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error

def objective(trial):
    cat_params = dict(
        iterations = trial.suggest_int("iterations", 100, 1000),
        learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth = trial.suggest_int("depth", 4, 10),
        l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bagging_temperature = trial.suggest_float('bagging_temperature', 0, 1.5),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        task_type='GPU',
        early_stopping_rounds=200,
        verbose=False
    )
    
    model = CatBoostRegressor(**cat_params)
    X_train_pool = Pool(X_train, y_train)
    X_valid_pool = Pool(X_val, y_val)
    model.fit(X=X_train_pool, eval_set=X_valid_pool)
    
    y_pred = model.predict(X_val)
    score = mean_squared_error(y_val, y_pred, squared=False)
    
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
best_params = study.best_params


[I 2024-12-28 11:06:38,436] A new study created in memory with name: no-name-741a8b0d-4020-4bc7-8662-e2d814da7459
[I 2024-12-28 11:06:47,647] Trial 0 finished with value: 0.11500392253216923 and parameters: {'iterations': 764, 'learning_rate': 0.009303072596529521, 'depth': 4, 'l2_leaf_reg': 0.0023531561128510253, 'bagging_temperature': 0.0673928393298992, 'random_strength': 8.488583276839597e-07}. Best is trial 0 with value: 0.11500392253216923.
[I 2024-12-28 11:06:51,608] Trial 1 finished with value: 0.1468884900640772 and parameters: {'iterations': 416, 'learning_rate': 0.0013233325617292848, 'depth': 4, 'l2_leaf_reg': 1.3053907445783347e-05, 'bagging_temperature': 0.7554177136349179, 'random_strength': 1.651724268879777e-07}. Best is trial 0 with value: 0.11500392253216923.
[I 2024-12-28 11:07:03,691] Trial 2 finished with value: 0.1063854525132171 and parameters: {'iterations': 539, 'learning_rate': 0.028025889933093816, 'depth': 6, 'l2_leaf_reg': 0.038665831252216495, 'bagging_te

fitting the model.

In [17]:
best_model = CatBoostRegressor(**best_params)
X_train_pool = Pool(X_train, y_train)
X_valid_pool = Pool(X_val, y_val)
best_model.fit(X=X_train_pool, eval_set=X_valid_pool)

0:	learn: 0.1636584	test: 0.1581871	best: 0.1581871 (0)	total: 156ms	remaining: 2m 27s
1:	learn: 0.1595281	test: 0.1546027	best: 0.1546027 (1)	total: 209ms	remaining: 1m 38s
2:	learn: 0.1552966	test: 0.1509522	best: 0.1509522 (2)	total: 385ms	remaining: 2m
3:	learn: 0.1514875	test: 0.1477639	best: 0.1477639 (3)	total: 549ms	remaining: 2m 9s
4:	learn: 0.1483397	test: 0.1450637	best: 0.1450637 (4)	total: 627ms	remaining: 1m 57s
5:	learn: 0.1451534	test: 0.1423564	best: 0.1423564 (5)	total: 781ms	remaining: 2m 2s
6:	learn: 0.1424766	test: 0.1401265	best: 0.1401265 (6)	total: 936ms	remaining: 2m 5s
7:	learn: 0.1397710	test: 0.1378470	best: 0.1378470 (7)	total: 1.09s	remaining: 2m 7s
8:	learn: 0.1373776	test: 0.1359332	best: 0.1359332 (8)	total: 1.25s	remaining: 2m 9s
9:	learn: 0.1352043	test: 0.1342051	best: 0.1342051 (9)	total: 1.4s	remaining: 2m 11s
10:	learn: 0.1331643	test: 0.1324546	best: 0.1324546 (10)	total: 1.55s	remaining: 2m 11s
11:	learn: 0.1314045	test: 0.1310611	best: 0.131061

In [ ]:
'''import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    xgb_params = dict(
        n_estimators=trial.suggest_int("n_estimators", 100, 1000),  # Similar to iterations
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        max_depth=trial.suggest_int("max_depth", 3, 10),
        min_child_weight=trial.suggest_float("min_child_weight", 1e-3, 10.0, log=True),
        subsample=trial.suggest_float("subsample", 0.5, 1.0),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.5, 1.0),
        gamma=trial.suggest_float("gamma", 0.0, 5.0),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),  # L1 regularization
        reg_lambda=trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),  # L2 regularization
        random_state=42,
        tree_method="gpu_hist",  # Use GPU if available for faster training
        early_stopping_rounds=200
    )
    
    model = XGBRegressor(**xgb_params)
    
    # Fit the model
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
        verbose=False
    )
    
    
    y_pred = model.predict(X_val)
    score = mean_squared_error(y_val, y_pred, squared=False)  
    
    return score


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
best_params = study.best_params
print("Best parameters:", best_params)'''


In [ ]:
'''best_model = XGBRegressor(**best_params)


best_model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_val, y_val)], 
    eval_metric="rmse", 
    early_stopping_rounds=200,  
    verbose=True  
)'''

here's the test data.

In [18]:
test_file_path = '/kaggle/input/bitfest-datathon-2025/test.csv'
test_df = pd.read_csv(test_file_path)


test_df.fillna('Unknown', inplace=True)


test_df['passing_years'] = test_df['passing_years'].apply(lambda x: x[0] if isinstance(x, list) else x)

X_test_final = test_df.drop('ID', axis=1)  


X_test_final = preprocessor.transform(X_test_final)

predicting and saving the csv file.

In [19]:
results = best_model.predict(X_test_final)
submission = pd.DataFrame({'ID': test_df['ID'], 'matched_score': results})
submission.to_csv('bitfest5.csv', index=False)